# JSON-to-MySQL Movie Database

The JSON-to-MySQL Movie Database project involves creating a relational database system using MySQL to store comprehensive information about movies, actors, directors, genres, and their relationships. The project also includes developing a Python script for uploading data from JSON files into the MySQL database.

Tables:

- ACTORS: This table stores information about actors.
- DIRECTORS: Contains data about movie directors.
- DIRECTORS_GENRES: A linking table that represents the many-to-many relationship between directors and genres.
- MOVIES: Stores details about movies, including their unique IDs, titles, year, and rakingss.
- MOVIES_GENRES: A linking table connecting movies with genres.
- ROLES: This table contains information about the roles actors play in movies.
- MOVIES_DIRECTORS: Another linking table that establishes the relationship between movies and directors.

Following the next ER DIAGRAM:

![IMDB](IMDB.png)

## Import library

In [1]:
import os 
import pandas
import json
import mysql.connector
from tqdm import tqdm

## Create Database and verification

In [2]:
conn = mysql.connector.connect(host = "localhost",
                                 user = "root",
                                 passwd = "jsonfile")  
cur = conn.cursor()
#creating a new database  
cur.execute("CREATE DATABASE IMDBJSON")  # TO CREATE A NEW DATABASE UNCOMENT THIS LINE  
#getting the list of all the databases which will now include the new database PythonDB  
db = cur.execute("SHOW DATABASES")   
for x in cur:  
    print(x)  # verify database is created 
conn.close() # Close Conection 

('imdbjson',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('task',)
('world',)


## Create tables based on ER SCHEMA

In [3]:

conn = mysql.connector.connect(
      host='localhost', 
      user='root', 
      password='jsonfile', 
      database='imdbjson'
   )
cur = conn.cursor()
   #Creating Directors table 
sql ='''CREATE TABLE DIRECTORS (
         ID INT(11) NOT NULL DEFAULT 0,
         FIRST_NAME VARCHAR(100) DEFAULT NULL,
         LAST_NAME  VARCHAR(100) DEFAULT NULL,
         PRIMARY KEY (ID),
         KEY DIRECTORS_FIRST_NAME (FIRST_NAME),
         KEY DIRECTORS_LAST_NAME (LAST_NAME)
         )ENGINE=INNODB;
         '''
cur.execute(sql)
   #Creating actors table 
sql ='''CREATE TABLE ACTORS (
         ID INT(11) NOT NULL DEFAULT 0,
         FIRST_NAME VARCHAR(100) DEFAULT NULL,
         LAST_NAME VARCHAR(100) DEFAULT NULL,
         GENDER CHAR(1) DEFAULT NULL,
         PRIMARY KEY (ID),
         KEY ACTORS_FIRST_NAME (FIRST_NAME),
         KEY ACTORS_LAST_NAME (LAST_NAME)
         )ENGINE=INNODB;
         '''
cur.execute(sql)
   #Creating movies table 
sql ='''CREATE TABLE MOVIES (
         ID INT(11) NOT NULL DEFAULT 0,
         NAME VARCHAR(100) DEFAULT NULL,
         YEAR INT(11) DEFAULT NULL,
         RANKED FLOAT DEFAULT NULL,
         PRIMARY KEY (ID),
         KEY MOVIES_NAME (NAME)
         )ENGINE=INNODB;
         '''
cur.execute(sql)
   #Creating directors genres table 
sql ='''CREATE TABLE DIRECTORS_GENRES (
         DIRECTOR_ID INT(11) NOT NULL,
         GENRE VARCHAR(100) NOT NULL,
         PROB FLOAT DEFAULT NULL,
         PRIMARY KEY (DIRECTOR_ID,GENRE),
         KEY DIRECTOR_GENRES_DIRECTOR_ID (DIRECTOR_ID),
         CONSTRAINT `DIRECTORS_GENRES__1` FOREIGN KEY (DIRECTOR_ID) REFERENCES DIRECTORS (ID) 
         ON DELETE CASCADE ON UPDATE CASCADE
         ) ENGINE=INNODB;
         '''
cur.execute(sql)
   #Creating movies director table 
sql ='''CREATE TABLE MOVIES_DIRECTORS (
         DIRECTOR_ID INT(11) NOT NULL,
         MOVIE_ID INT(11) NOT NULL,
         PRIMARY KEY (DIRECTOR_ID,MOVIE_ID),
         KEY MOVIES_DIRECTORS_DIRECTOR_ID (DIRECTOR_ID),
         KEY MOVIES_DIRECTORS_MOVIE_ID (MOVIE_ID),
         CONSTRAINT `MOVIES_DIRECTORS__1` FOREIGN KEY (DIRECTOR_ID) REFERENCES DIRECTORS (ID) 
         ON DELETE CASCADE ON UPDATE CASCADE,
         CONSTRAINT `MOVIES_DIRECTORS__2` FOREIGN KEY (MOVIE_ID) REFERENCES MOVIES (ID) 
         ON DELETE CASCADE ON UPDATE CASCADE
         ) ENGINE=INNODB;
         '''
cur.execute(sql)
   #Creating roles  table 
sql ='''CREATE TABLE ROLES (
         ACTOR_ID INT(11) NOT NULL,
         MOVIE_ID INT(11) NOT NULL,
         ROLE VARCHAR(100) NOT NULL,
         PRIMARY KEY (ACTOR_ID,MOVIE_ID,ROLE),
         KEY ACTOR_ID (ACTOR_ID),
         KEY MOVIE_ID (MOVIE_ID),
         CONSTRAINT ROLES__1 FOREIGN KEY (ACTOR_ID) REFERENCES ACTORS (ID)
         ON DELETE CASCADE ON UPDATE CASCADE,
         CONSTRAINT ROLES__2 FOREIGN KEY (MOVIE_ID) REFERENCES MOVIES (ID) 
         ON DELETE CASCADE ON UPDATE CASCADE
         ) ENGINE=INNODB;
         '''
cur.execute(sql)
   #Creating movies genres table 
sql ='''CREATE TABLE MOVIES_GENRES (
         MOVIE_ID INT(11) NOT NULL,
         GENRE VARCHAR(100) NOT NULL,
         PRIMARY KEY (MOVIE_ID,GENRE),
         KEY MOVIE_GENRES_MOVIE_ID (MOVIE_ID),
         CONSTRAINT MOVIES_GENRES__1 FOREIGN KEY (MOVIE_ID) REFERENCES MOVIES (ID)
         ON DELETE CASCADE ON UPDATE CASCADE
         ) ENGINE=INNODB;
         '''
cur.execute(sql)



## Upload JSON files provided to each table 

In [4]:
# Open json file for actors table
with open ('actors.json','r') as f :
    actors = json.load(f)
# Upload json information for actors table to MySQL    
print("Actors table uploading")
for item in tqdm(actors,colour="yellow") : 
    i=(item["id"],item["first_name"],item["last_name"],item["gender"])
    sql= """INSERT into ACTORS (ID,FIRST_NAME, LAST_NAME,GENDER) VALUES(%s,%s,%s,%s)"""
    cur.execute(sql,i)
    conn.commit()
    
# Open json file for directors table
with open ('directors.json','r') as f :
    directors = json.load(f)
# Upload json information for directors table to MySQL    
print("Directors table uploading")
for item in tqdm(directors,colour="blue") : 
    i=(item["id"],item["first_name"],item["last_name"])
    sql= """INSERT into DIRECTORS (ID,FIRST_NAME, LAST_NAME) VALUES(%s,%s,%s)"""
    cur.execute(sql,i)
    conn.commit()
    
# Open json file for movies table
with open ('movies.json','r') as f :
    movies = json.load(f)
# Upload json information for movies table to MySQL    
print("Movies table uploading")
for item in tqdm(movies,colour="red") : 
    i=(item["id"],item["name"],item["year"],item["rank"])
    sql= """INSERT into MOVIES (ID,NAME,YEAR,RANKED) VALUES(%s,%s,%s,%s)"""
    cur.execute(sql,i)
    conn.commit()

# Open json file for directors genres table
with open ('directorsgenres.json','r') as f :
    directorsgenres = json.load(f)
# Upload json information for director genres table to MySQL    
print("Directors Genres table uploading")
for item in tqdm(directorsgenres,colour="yellow") : 
    i=(item["director_id"],item["genre"],item["prob"])
    sql= """INSERT into DIRECTORS_GENRES (DIRECTOR_ID,GENRE, PROB) VALUES(%s,%s,%s)"""
    cur.execute(sql,i)
    conn.commit()

# Open json file for movies genres table
with open ('moviesgenres.json','r') as f :
    moviesgenres = json.load(f)
# Upload json information for movies genres table to MySQL    
print("Movies Genres table uploading")
for item in tqdm(moviesgenres,colour="blue") : 
    i=(item["movie_id"],item["genre"])
    sql= """INSERT into MOVIES_GENRES (MOVIE_ID, GENRE) VALUES(%s,%s)"""
    cur.execute(sql,i)
    conn.commit()
    
# Open json file for movies directors table
with open ('moviesdirectors.json','r') as f :
    moviesdirectors = json.load(f)
# Upload json information for movies directors table to MySQL    
print("Movies Directors table uploading")
for item in tqdm(moviesdirectors,colour="red") : 
    i=(item["director_id"],item["movie_id"])
    sql= """INSERT into MOVIES_DIRECTORS (DIRECTOR_ID, MOVIE_ID) VALUES(%s,%s)"""
    cur.execute(sql,i)
    conn.commit()
    
# Open json file for roles table
with open ('roles.json','r') as f :
    roles = json.load(f)
# Upload json information for roles table to MySQL    
print("Roles table uploading")
for item in tqdm(roles,colour="green") : 
    i=(item["actor_id"],item["movie_id"],item["role"])
    sql= """INSERT into ROLES (ACTOR_ID,MOVIE_ID,ROLE) VALUES(%s,%s,%s)"""
    cur.execute(sql,i)
    conn.commit()

Actors table uploading


100%|██████████| 10000/10000 [00:20<00:00, 483.35it/s]


Directors table uploading


100%|██████████| 10000/10000 [00:19<00:00, 503.95it/s]


Movies table uploading


...

100%|██████████| 10136/10136 [00:20<00:00, 501.79it/s]


Movies Directors table uploading


100%|██████████| 872/872 [00:01<00:00, 510.77it/s]


Roles table uploading


100%|██████████| 1139/1139 [00:02<00:00, 418.95it/s]


## Verify Number of row upload to each table

In [5]:
#Verify data was uploaded
tables=["ACTORS" , "DIRECTORS" , "DIRECTORS_GENRES", "MOVIES" , "MOVIES_GENRES" ,"ROLES","MOVIES_DIRECTORS"]
conn = mysql.connector.connect(
      host='localhost', 
      user='root', 
      password='jsonfile', 
      database='imdbjson'
   )
cur = conn.cursor()
for table in tables : 
   cur.execute(f"SELECT COUNT(*) FROM {table}") 
   result = cur.fetchone()
   row_count = result[0]
   print("Number of rows in the table:", table," ",row_count)
conn.close()

Number of rows in the table: ACTORS   10000
Number of rows in the table: DIRECTORS   10000
Number of rows in the table: DIRECTORS_GENRES   17376
Number of rows in the table: MOVIES   10000
Number of rows in the table: MOVIES_GENRES   10136
Number of rows in the table: ROLES   1139
Number of rows in the table: MOVIES_DIRECTORS   872
